##### Importações:

In [29]:
import logging
from selenium.webdriver.remote.remote_connection import LOGGER
from selenium.common.exceptions import *
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from parsel import Selector
from time import sleep
import random
import pandas as pd

##### Funções

Função que inicia o driver

In [11]:
def start_driver():
    options = Options()
    LOGGER.setLevel(logging.WARNING)
    # add arguments to options variable:
    arguments = ['--lang=pt-BR', '--window-size=1920,1080'] #'--headless'
    for argument in arguments:
        options.add_argument(argument)

    driver = webdriver.Chrome(options=options)

    wait = WebDriverWait(
        driver,
        10,
        poll_frequency=1,
        ignored_exceptions=[
            NoSuchElementException,
            ElementNotVisibleException,
            ElementNotSelectableException
        ]
    )
    return driver, wait

Função para digitar naturalmente

In [12]:
def natural_type(text, field):
    for letter in text:
        field.send_keys(letter)
        sleep(random.randint(1,5)/30)


Função para abrir a página

In [13]:
def get_page(site, driver, wait):
    driver.get(site)
    wait.until(
        expected_conditions.visibility_of_all_elements_located(
            (By.XPATH, "//input[@id='text-input-what']")
        )
    )    

Função para pesquisar as vagas

In [14]:
def search(driver, text):
    input_field = driver.find_element(By.XPATH, "//input[@id='text-input-what']")
    natural_type(text, input_field)
    sleep(2)
    input_field.send_keys(Keys.ENTER)

##### Executando...

In [32]:
site = "https://br.indeed.com"
text = "Python"

In [16]:
driver, wait = start_driver()

In [17]:
get_page(site, driver, wait)

In [18]:
search(driver, text)

Scraping testes

In [19]:
response = Selector(text=driver.page_source)

Achando os cargos:

In [20]:
# xpath das divs com as informações:
"//td/div[@class='css-dekpa e37uo190']"

"//td/div[@class='css-dekpa e37uo190']"

In [21]:
info = response.xpath("//td[@class='resultContent css-1qwrrf0 eu4oa1w0']")

In [22]:
info[0].xpath("./div/h2/a/span/text()").get()

'Engenheira(o) de Software - Centro IA'

In [23]:
for cargo in info:
    print(cargo.xpath("./h2/a/span/text()").get())

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


Achando os **nomes** da empresa:

xpath = //div[@class='company_location css-17fky0v e37uo190']/div[@data-testid='text-location']/text()

In [24]:
info[2].xpath("./div[@class='company_location css-17fky0v e37uo190']//span[@data-testid='company-name']/text()").get()

'Netvagas'

Achando os **locais** das vagas:

In [25]:
info[1].xpath("./div[@class='company_location css-17fky0v e37uo190']/div/div[@data-testid='text-location']/text()").get()

Achando os **links** das vagas:

In [26]:
link = site + info[1].xpath("./div/h2/a/@href").get()
print(link)

https://br.indeed.com//pagead/clk?mo=r&ad=-6NYlbfkN0CiU16kVcsoWM1vVFh8O9FB-UbFnlP1pxxBBYLKro3yC46KxQASDEF2Uf2INevJK6cnury13WdNlPc99A5b61Hy6c51j8jpavDBDTsAm79lACDW8P_tSYn02t5bcgD9GX9t0i2ofr8Nsew46P0aRGIskVrH18klGSjD0upUpIIYSViOG53hv-gFdMvLy8s1O_1Aa7fNuTt5pa0X-U10uMA1xa95Nza17JRfptX1siuTpUOUx3t1Tplu2yH2p2PKQX202Dc3AvdKIqSHWE-62NCHG2zuZtVR5sIeuP4c_oY60rw7Cce0_VEdv4Y1BxcAKFPJI7KcHdvv0TNNWAoh48JUbCUD8VTMrIqUncCGh5y9XcjGoMwtQcpa9pf4wpSbwFQw8goc6MIryDo0Q3ubrEwT6PQXgnwQVwsr-McdTtmWdR9Hbg9iaamw_ySvxC3m6RaEBGTBDwLRAn8kb6RQSuftNgjnhADyo-jqIEWfq7ncXWRSepN0rKprofkfWbcIIlrnhmsnc-MJYJsimT8X2rL4xwySbuDApxzvLaK0KSXiyWsijY_ShVnto0zWUNb4qmm0yIgDAuwmStmxjUBiws0LfymlnRdzYhSjjFbsz28W3w==&xkcb=SoBn6_M3_M6yCWwGtZ0KbzkdCdPP&camk=nUmJqO2E8ri8cYTe25DeIg==&p=1&fvj=1&vjs=3


##### Função para scraping

In [27]:
def scrape(driver, wait, base_url):
    response = Selector(text=driver.page_source)
    for vaga in response.xpath("//td[@class='resultContent css-1qwrrf0 eu4oa1w0']"):
        yield {
            "Cargo" : vaga.xpath("./div/h2/a/span/text()").get(),
            "Empresa" : vaga.xpath("./div[@class='company_location css-17fky0v e37uo190']//span[@data-testid='company-name']/text()").get(),
            "Local" : vaga.xpath("./div[@class='company_location css-17fky0v e37uo190']/div/div[@data-testid='text-location']/text()").get(),
            "Link" : base_url + vaga.xpath("./div/h2/a/@href").get()
        }

##### Coletando dados de várias páginas:

In [ ]:
def main():
    

In [33]:
dados = pd.DataFrame(list(scrape(driver, wait, site)))

In [34]:
dados

,Cargo,Empresa,Local,Link
0,Engenheira(o) de Software - Centro IA,Dell,"Modelo Híbrido in São Paulo, SP",https://br.indeed.com/pagead/clk?mo=r&ad=-6NYl...
1,QA Analyst (with SQL),Global Vigilance - Harris Computer Guardian...,None,https://br.indeed.com/pagead/clk?mo=r&ad=-6NYl...
2,Pessoa desenvolvedora back-end python júnior (...,Netvagas,None,https://br.indeed.com/rc/clk?jk=a45bcc2536dbbd...
3,Pessoa Desenvolvedora Back-end Python Júnior (...,Gran,None,https://br.indeed.com/rc/clk?jk=10bcda8603838d...
4,Pessoa Desenvolvedora Python - Trabalho Remoto,BairesDev,"Home Office in Porto Alegre, RS",https://br.indeed.com/rc/clk?jk=542931b567264d...
5,Desenvolvedor Python,PF,"Mato Grosso, MT",https://br.indeed.com/rc/clk?jk=4949794b3451ef...
6,Desenvolvedor Python Pleno,ONR,"Home Office in São Paulo, SP",https://br.indeed.com/rc/clk?jk=23b6bdf3cc5435...
7,Desenvolvedor Backend Júnior Python / Pesquisa...,BairesDev,"Home Office in Campinas, SP",https://br.indeed.com/rc/clk?jk=113cea2de99293...
8,Administrador de Banco de Dados DBA (Infraestr...,Cresol Oficial,"Florianópolis, SC",https://br.indeed.com/pagead/clk?mo=r&ad=-6NYl...
9,Python Data Scientist,Aera Technology & Consulting,None,https://br.indeed.com/rc/clk?jk=ba0ba746196c3d...
